In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import cvxpy as cp
from IPython.display import display
from typing import Dict, Any, Tuple

class ProductionSystem:
    """Randomly generates convex generators for the dispatch study."""

    def __init__(self, n_machines=6, seed=7,
                 alpha_bounds=(0.05, 0.2), beta_bounds=(2.0, 5.0),
                 gamma_bounds=(10.0, 20.0), capacity_bounds=(50.0, 150.0)):
        rng = np.random.default_rng(seed)
        self.n = n_machines
        self.alpha = rng.uniform(*alpha_bounds, n_machines)
        self.beta = rng.uniform(*beta_bounds, n_machines)
        self.gamma = rng.uniform(*gamma_bounds, n_machines)
        self.l = np.zeros(n_machines)
        self.u = rng.uniform(*capacity_bounds, n_machines)

    def describe(self) -> pd.DataFrame:
        machines = np.arange(1, self.n + 1)
        data = {
            "alpha": self.alpha,
            "beta": self.beta,
            "gamma": self.gamma,
            "min_cap": self.l,
            "max_cap": self.u,
        }
        return pd.DataFrame(data, index=machines).rename_axis("machine")

    def total_capacity(self) -> float:
        return float(np.sum(self.u))


def solve_dispatch(system: ProductionSystem, mu_D: float, sigma_D: float,
                   reliability: float = 0.95, mode: str = "normal") -> dict:
    if sigma_D < 0 or mu_D < 0:
        raise ValueError("Demand statistics must be non-negative")
    if not 0.0 < reliability < 1.0:
        raise ValueError("Reliability must lie in (0, 1)")

    x = cp.Variable(system.n)
    alpha_risk = 1.0 - reliability

    if mode == "normal":
        z_score = stats.norm.ppf(reliability)
        D_eff = mu_D + z_score * sigma_D
    elif mode == "robust":
        k_robust = np.sqrt((1 - alpha_risk) / alpha_risk)
        D_eff = mu_D + k_robust * sigma_D
    else:
        raise ValueError("Mode must be 'normal' or 'robust'")

    if D_eff > system.total_capacity() + 1e-6:
        raise ValueError("Effective demand exceeds installed capacity")

    objective = cp.Minimize(cp.sum(cp.multiply(system.alpha, x**2) + cp.multiply(system.beta, x)))
    constraints = [cp.sum(x) >= D_eff, x <= system.u, x >= system.l]

    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.OSQP, warm_start=True)

    if prob.status not in {"optimal", "optimal_inaccurate"}:
        return {"status": prob.status}

    dispatch = x.value
    total_cost = float(np.sum(system.alpha * dispatch**2 + system.beta * dispatch + system.gamma))

    return {
        "status": prob.status,
        "x": dispatch,
        "cost": total_cost,
        "objective": prob.value,
        "D_eff": float(D_eff),
        "lambda": constraints[0].dual_value,
        "nu_u": constraints[1].dual_value,
        "nu_l": constraints[2].dual_value,
    }


def summarize_dispatch(system: ProductionSystem, res: dict) -> pd.DataFrame:
    if res.get("status") not in {"optimal", "optimal_inaccurate"}:
        raise ValueError("Cannot summarize infeasible solution")

    dispatch = res["x"]
    table = system.describe().copy()
    table["dispatch"] = dispatch
    table["upper_slack"] = system.u - dispatch
    table["lower_slack"] = dispatch - system.l
    table["nu_upper"] = res["nu_u"]
    table["nu_lower"] = res["nu_l"]
    table["marginal_cost"] = 2 * system.alpha * dispatch + system.beta
    table["incremental_cost"] = system.alpha * dispatch**2 + system.beta * dispatch + system.gamma
    return table

In [ ]:
system = ProductionSystem(n_machines=6, seed=7)
mu_D = 350.0
sigma_D = 40.0
reliability = 0.95

res_norm = solve_dispatch(system, mu_D, sigma_D, reliability=reliability, mode="normal")
res_rob = solve_dispatch(system, mu_D, sigma_D, reliability=reliability, mode="robust")
summary_norm = summarize_dispatch(system, res_norm)
summary_rob = summarize_dispatch(system, res_rob)

print("=== Normal (Gaussian) Scenario ===")
print(f"Effective demand: {res_norm['D_eff']:.1f} MW | Total cost: ${res_norm['cost']:.2f}")
display(summary_norm)

print("\n=== Robust (Chebyshev) Scenario ===")
print(f"Effective demand: {res_rob['D_eff']:.1f} MW | Total cost: ${res_rob['cost']:.2f}")
display(summary_rob)

# Preserve common aliases used by downstream plotting helpers
sys = system
MU = mu_D
SIGMA = sigma_D

In [ ]:
def plot_allocation(system, res, title="Optimal Resource Allocation"):
    """
    Visualizes how much of each machine's capacity is utilized.
    """
    indices = np.arange(system.n)
    
    plt.figure(figsize=(10, 5))
    
    # 1. Plot the Max Capacity (The Limit)
    plt.bar(indices, system.u, color='lightgray', label='Unused Capacity', hatch='//', edgecolor='gray')
    
    # 2. Plot the Optimal Production (The Decision)
    # Color code: Green if maxed out, Blue if partial, Red if zero
    colors = []
    for i, val in enumerate(res['x']):
        if val >= system.u[i] - 1e-3: colors.append('forestgreen') # Maxed
        elif val <= 1e-3: colors.append('lightcoral') # Unused
        else: colors.append('royalblue') # Partial
            
    plt.bar(indices, res['x'], color=colors, label='Production Output')
    
    # Formatting
    plt.xlabel('Machine ID')
    plt.ylabel('Production Units')
    plt.title(title)
    plt.legend(loc='upper right')
    
    # Intuitive Annotation
    plt.text(0, system.u[0] + 5, "Green = Maxed Out (Efficient)", fontsize=9, color='forestgreen')
    plt.show()

# RUN IT
# plot_allocation(sys, res_norm, "Scenario: Normal Demand (95% Reliability)")

In [32]:
def plot_comparison(res_norm, res_rob):
    labels = ['Normal Model\n(Assumes Gaussian)', 'Robust Model\n(Distribution Free)']
    costs = [res_norm['cost'], res_rob['cost']]
    targets = [res_norm['D_eff'], res_rob['D_eff']]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot 1: Cost Comparison
    bars = ax1.bar(labels, costs, color=['blue', 'darkred'], alpha=0.7)
    ax1.set_ylabel('Total Operating Cost ($)')
    ax1.set_title('Financial Impact of Robustness')
    
    # Add text on top of bars
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'${height:,.0f}', ha='center', va='bottom', fontsize=12, fontweight='bold')
        
    # Plot 2: Safety Buffer Comparison
    ax2.bar(labels, targets, color=['skyblue', 'salmon'], alpha=0.7)
    ax2.set_ylabel('Required Production Target (Units)')
    ax2.set_title('Safety Buffer Requirements')
    
    # Draw the "Gap"
    gap = res_rob['D_eff'] - res_norm['D_eff']
    ax2.annotate(f'+{gap:.1f} Units\nExtra Buffer', 
                 xy=(1, res_rob['D_eff']), xytext=(0.5, res_rob['D_eff']),
                 arrowprops=dict(facecolor='black', arrowstyle='->'))

    plt.tight_layout()
    plt.show()

# RUN IT
# plot_comparison(res_norm, res_rob)

In [33]:
def plot_sensitivity(system, mu, sigma):
    alphas = [0.10, 0.05, 0.02, 0.01, 0.005] # 90% to 99.5%
    reliabilities = [(1-a)*100 for a in alphas]
    costs = []
    
    for a in alphas:
        # Run the solver for each alpha
        res = solve_dispatch(system, mu, sigma, reliability=(1-a), mode='normal')
        costs.append(res['cost'])
        
    plt.figure(figsize=(8, 5))
    plt.plot(reliabilities, costs, marker='o', linewidth=2, color='purple')
    
    plt.xlabel('Target Reliability (%)')
    plt.ylabel('Total Cost ($)')
    plt.title('Sensitivity Analysis: The Cost of Perfection')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Highlight the "Knee" of the curve
    plt.annotate('Diminishing Returns\n(Cost explodes here)', 
                 xy=(99.0, costs[-2]), xytext=(94, costs[-1]),
                 arrowprops=dict(facecolor='black', shrink=0.05))
    
    plt.show()

# RUN IT
# plot_sensitivity(sys, MU, SIGMA)

In [ ]:
plot_allocation(system, res_norm, "Scenario: Normal Demand (95% Reliability)")
plot_allocation(system, res_rob, "Scenario: Robust Chebyshev Hedge")
plot_comparison(res_norm, res_rob)
plot_sensitivity(system, mu_D, sigma_D)